# Scraper institutes
This notebook is a monthly scraper used to retrieve information about condition in detention centers in Italy. To do so, it uses the id numbers of the various detention centers to navigate to the dedicated webpages with Selenium, store locally the html code of the page and then parse it using BeautifuSoup. The information is then stored in a pandas dataframe and saved as a csv file.

In [1]:
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup
import asyncio
from playwright.async_api import async_playwright
from time import sleep
from pathlib import Path

In [2]:
# Get current date
current_month = datetime.datetime.now().strftime("%Y-%m")
current_day = datetime.datetime.now().strftime("%Y-%m-%d")

In [3]:
# Collect institutes id numbers
df_institutes = pd.read_csv(f'../outputs/clean/institutes_info.csv')
prison_ids = df_institutes['id_istituto'].tolist()

In [4]:
# Function to grab the html code of the page
async def get_html(prison_id, current_day):

    # dest = Path(f"../outputs/raw/snapshots/{current_day}_{prison_id}.html")

    # if dest.exists() : #... load it from file
    #     print(f"Already have {dest}, loading!")
    #     page_html = open(dest).read()
    # else:


    try:
        await page.goto(f"{BASE_URL}{prison_id}")
        print("Fetching " + f"{BASE_URL}{prison_id}")
        page_html = await page.content()
        
        # Stores html code in dest
        # dest.write_text(page_html)
    finally:
        await browser.close()
        await playwright.stop()
    
    return page_html

In [5]:
# Function to extract institute name and type
def extract_institute_details(soup):
    institute_name = soup.find('h1', {'class': 'titoloIstituto'}).text.strip()
    institute_type = soup.find('h3', {'class': 'titoloIstituto'}).text.strip()
    return institute_name, institute_type

# Function to extract capacity table data
def extract_capacity_data(soup):
    table_capienze = soup.find_all('table')[0]
    rows = table_capienze.find_all('tr')
    if len(rows) > 1:  # Ensure there are rows in the table
        cells = rows[1].find_all('td') 
        posti_regolamentari = int(cells[0].text.strip())
        posti_non_disponibili = int(cells[1].text.strip())
        totale_detenuti = int(cells[2].text.strip())
    else:
        posti_regolamentari = posti_non_disponibili = totale_detenuti = 0
    return posti_regolamentari, posti_non_disponibili, totale_detenuti

# Function to extract updated date
def extract_updated_date(soup, header_text):
    target_span = soup.find('h2', text=header_text)
    if target_span:
        span = target_span.find_next_sibling('span')
        return span.text.strip() if span else 'NA'
    return 'NA'

def extract_personnel_details(soup, header_text):
    target_span = soup.find('h2', text=header_text)
    if target_span:
        try:
            div = target_span.find_next('div', {'class': 'listaContenutiComplessi'})
            spans = div.find_all('span', {'class': 'valoreSottocampo'})
            return spans[0].text.strip(), spans[1].text.strip(), spans[2].text.strip()
        except:
            return 'NA', 'NA', 'NA'
    return 'NA', 'NA', 'NA'

# Function to extract staff table data
def extract_staff_data(soup):
    table_staff = soup.find_all('table')[1]
    cells = table_staff.find_all('td')
    polizia_penitenziaria_effettivi = int(cells[0].text.strip())
    polizia_penitenziaria_previsti = int(cells[1].text.strip())
    amministrativi_effettivi = int(cells[2].text.strip())
    amministrativi_previsti = int(cells[3].text.strip())
    educatori_effettivi = int(cells[4].text.strip())
    educatori_previsti = int(cells[5].text.strip())
    return (polizia_penitenziaria_effettivi, polizia_penitenziaria_previsti,
            amministrativi_effettivi, amministrativi_previsti,
            educatori_effettivi, educatori_previsti)

# Function to extract last update date
def extract_date_of_last_update(soup):
    # Police staff
    target_span= soup.find('h2', text='personale polizia penitenziaria aggiornato al')
    try:
        span = target_span.find_next_sibling('span')
        personale_polizia_aggiornato_al = span.text.strip()
    except:
        personale_polizia_aggiornato_al = 'NA'

    # Administrative staff
    target_span= soup.find('h2', text='personale amministrativo aggiornato al')
    try:
        span = target_span.find_next_sibling('span')
        personale_amministrativo_aggiornato_al = span.text.strip()
    except:
        personale_amministrativo_aggiornato_al = 'NA'

    return personale_polizia_aggiornato_al, personale_amministrativo_aggiornato_al
    

In [6]:
# Function to extract institute data
def get_prison_data(soup, current_day):

    institute_name, institute_type = extract_institute_details(soup)
    posti_regolamentari, posti_non_disponibili, totale_detenuti = extract_capacity_data(soup)
    dati_aggiornati_al = extract_updated_date(soup, 'dati aggiornati al ')
    asl, first_name_asl, last_name_asl = extract_personnel_details(soup, 'Responsabile ASL per il carcere')
    first_name, last_name, role = extract_personnel_details(soup, 'Direttore')
    (polizia_penitenziaria_effettivi, polizia_penitenziaria_previsti, amministrativi_effettivi, amministrativi_previsti, educatori_effettivi, educatori_previsti) = extract_staff_data(soup)
    personale_polizia_aggiornato_al, personale_amministrativo_aggiornato_al = extract_date_of_last_update(soup)

    prison_data = {
            'id': prison_id,
            'nome': institute_name,
            'tipo': institute_type,
            'posti_regolamentari': posti_regolamentari,
            'posti_non_disponibili': posti_non_disponibili,
            'posti_occupati': totale_detenuti,
            'posti_aggiornati_al': dati_aggiornati_al,
            'asl': asl,
            'nome_responsabile_asl': first_name_asl,
            'cognome_responsabile_asl': last_name_asl,
            'nome_direttore': first_name,
            'cognome_direttore': last_name,
            'ruolo_direttore': role,
            'personale_polizia_effettivi': polizia_penitenziaria_effettivi,
            'personale_polizia_previsti': polizia_penitenziaria_previsti,
            'personale_amministrativi_effettivi': amministrativi_effettivi,
            'personale_amministrativi_previsti': amministrativi_previsti,
            'personale_educatori_effettivi': educatori_effettivi,
            'personale_educatori_previsti': educatori_previsti,
            'personale_polizia_aggiornato_a': personale_polizia_aggiornato_al,
            'personale_amministrativo_aggiornato_al': personale_amministrativo_aggiornato_al,
        }
        
    return prison_data

In [ ]:
data = []

BASE_URL = "https://www.giustizia.it/giustizia/it/dettaglio_scheda.page?s="
playwright = await async_playwright().start()
browser = await playwright.firefox.launch()
context = await browser.new_context(viewport={'width': 1280, 'height': 800})
page = await context.new_page()



for prison_id in prison_ids:
    success = False
    for attempt in range(5):
        try:
            html_content = await get_html(prison_id, current_month)
            # Parse the html with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            prison_data = get_prison_data(soup, current_day)

            # Append prison_data to data list
            data.append(prison_data)

            success = True
            break  # Break the retry loop if successful

        except Exception as e:
            print(f"Attempt {attempt+1} failed for prison_id {prison_id}. Error: {e}")
            if attempt < 5:  # If not the last attempt, sleep for 10 seconds before retrying
                print("Reinitializing browser...")
                await browser.close()
                sleep(10)
                browser = await playwright.firefox.launch()
                context = await browser.new_context(viewport={'width': 1280, 'height': 800})
                page = await context.new_page()


    if not success:
        print(f"Failed to fetch data for prison_id {prison_id} after 5 attempts.")
    # Sleep for 5 seconds before making the next request
    sleep(5)

# Convert prison_data_list to a Pandas DataFrame
data_df = pd.DataFrame(data)

In [ ]:
len(data_df)

In [15]:
old_data_path = Path('../outputs/raw/institutes_raw.csv')

if old_data_path.exists():
  old_data = pd.read_csv('../outputs/raw/institutes_raw.csv')
  combined_data = pd.concat([old_data, data_df], ignore_index=True)
  combined_data.drop_duplicates(inplace=True)
else:
  combined_data = data_df
  combined_data.drop_duplicates(inplace=True)

combined_data.to_csv('../outputs/raw/institutes_raw.csv', index=False)

### Institutes - Totals

Some basic cleaning for the dates

In [ ]:
combined_data.head()

In [17]:
# Fixing dates
combined_data['posti_aggiornati_al'] = pd.to_datetime(combined_data['posti_aggiornati_al'], dayfirst=True)

combined_data['posti_aggiornati_al'] = combined_data['posti_aggiornati_al'].dt.strftime('%Y-%m-%d')

combined_data['personale_polizia_aggiornato_a'] = pd.to_datetime(combined_data['personale_polizia_aggiornato_a'], format='%d/%m/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
combined_data['personale_amministrativo_aggiornato_al'] = pd.to_datetime(combined_data['personale_amministrativo_aggiornato_al'], format='%d-%m-%Y', errors='coerce').dt.strftime('%Y-%m-%d')

In [ ]:
combined_data.tail()

In [19]:
combined_data.to_csv('../outputs/clean/institutes.csv', index=False)